# Importar modulos


In [1]:
using BilevelJuMP
using Ipopt

ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.


Crear una instancia del modelo con el optimizador ProductMode

In [2]:

model = BilevelModel(Ipopt.Optimizer; mode = BilevelJuMP.ProductMode(1e-9))

A JuMP Model
Feasibility problem with:
Variables: 0
Upper Constraints: 0
Lower Constraints: 0
Bilevel Model
Solution method: BilevelJuMP.ProductMode{Float64}(1.0e-9, false)
Solver name: Ipopt

Declarar variables del nivel superior


In [3]:
BilevelJuMP.@variable(Upper(model), x[i = 1:2], start = 0)

2-element Vector{BilevelJuMP.BilevelVariableRef}:
 x[1]
 x[2]

Variables del nivel inferior

In [4]:
BilevelJuMP.@variable(Lower(model), y[i = 1:2], start = -10)

2-element Vector{BilevelJuMP.BilevelVariableRef}:
 y[1]
 y[2]

Funcionon objetivo del nivel superior

In [21]:
BilevelJuMP.@objective(Upper(model), Min, 2x[1] + 2x[2] - 3y[1] - 3y[2] - 60)

2 x[1] + 2 x[2] - 3 y[1] - 3 y[2] - 60

Restricciones nivel superior

In [22]:
BilevelJuMP.@constraint(Upper(model), x[1] + x[2] + y[1] - 2y[2] - 40 <= 0)
BilevelJuMP.@constraint(Upper(model), [i = 1:2], x[i] >= 0)
BilevelJuMP.@constraint(Upper(model), [i = 1:2], x[i] <= 50)
BilevelJuMP.@constraint(Upper(model), [i = 1:2], y[i] >= -10)
BilevelJuMP.@constraint(Upper(model), [i = 1:2], y[i] <= 20)

2-element Vector{JuMP.ConstraintRef{BilevelModel, Int64, JuMP.ScalarShape}}:
 y[1] ≤ 20.0
 y[2] ≤ 20.0

Funcion objetivo nivel inferior

In [23]:
BilevelJuMP.@objective(Lower(model), Min, (-x[1] + y[1] + 40)^2 + (-x[2] + y[2] + 20)^2)

x[1]² - 2 y[1]*x[1] + y[1]² + x[2]² - 2 y[2]*x[2] + y[2]² - 80 x[1] + 80 y[1] - 40 x[2] + 40 y[2] + 2000

In [24]:
BilevelJuMP.@constraint(Lower(model), [i = 1:2], -x[i] + 2y[i] <= -10)
BilevelJuMP.@constraint(Lower(model), [i = 1:2], y[i] >= -10)
BilevelJuMP.@constraint(Lower(model), [i = 1:2], y[i] <= 20)

2-element Vector{JuMP.ConstraintRef{BilevelModel, Int64, JuMP.ScalarShape}}:
 y[1] ≤ 20.0
 y[2] ≤ 20.0

In [25]:
BilevelJuMP.optimize!(model)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       10
Number of nonzeros in inequality constraint Jacobian.:       42
Number of nonzeros in Lagrangian Hessian.............:        8

Total number of variables............................:       10
                     variables with only lower bounds:        2
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        4
Total number of equal

In [26]:
BilevelJuMP.primal_status(model)

FEASIBLE_POINT::ResultStatusCode = 1

In [27]:
BilevelJuMP.termination_status(model)

LOCALLY_SOLVED::TerminationStatusCode = 4

In [28]:
BilevelJuMP.objective_value(model)

-3.229015632655319e-8

In [30]:
BilevelJuMP.value.(x)

2-element Vector{Float64}:
 -8.747048220055642e-9
 -8.747048220161989e-9

In [31]:
BilevelJuMP.value(x[1])

-8.747048220055642e-9

In [16]:
x

2-element Vector{BilevelJuMP.BilevelVariableRef}:
 x[1]
 x[2]

In [32]:
for i in 1:length(x)
    println(BilevelJuMP.value(x[i]))
end

-8.747048220055642e-9
-8.747048220161989e-9


In [37]:
function write_response(model,x,y)
    # Escribir el resultado de la solucion primal
    p_s=BilevelJuMP.primal_status(model)
    println("Primal Status: $p_s")
    # Escribir el codigo y tipo de terminacion
    t_s=BilevelJuMP.termination_status(model)
    println("Termination Status: $t_s")
    # Objetive value
    o_v=BilevelJuMP.objective_value(model)
    println("Objetive Value $o_v")
    # Lider Vars Value
    l_vars=BilevelJuMP.value.(x)
    println("Lider Vars: $l_vars")
    # Follower Vars Value
    f_vars=BilevelJuMP.value.(y)
    println("Followers Vars Value $f_vars")
    
end
    

write_response (generic function with 2 methods)

In [ ]:
write_response(model,x,y)

Primal Status: FEASIBLE_POINT
Termination Status: LOCALLY_SOLVED
Objetive Value -3.229015632655319e-8
Lider Vars: [-8.747048220055642e-9, -8.747048220161989e-9]
Followers Vars Value [-10.000000000634344, -10.000000000265]
